In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("WATERTANKMASTER_COPY.xlsx", sheet_name = "2020-Present Dirty_renamedcols")

In [3]:
## pull out the lower manhattan tanks that are all smushed together
MHdf = df[(df["ZIP"] == 10013) | (df["ZIP"] == 10002) | (df["ZIP"] == 10280) | (df["ZIP"] == 10004) | (df["ZIP"] == 10041) | (df["ZIP"] == 10006)| (df["ZIP"] == 10005)| (df["ZIP"] == 10048) | (df["ZIP"] == 10200)| (df["ZIP"] == 10201)| (df["ZIP"] == 10202)]

In [4]:
MHdf.to_csv("Manhattan_WaterTanks.csv")

In [5]:
## summarize each row that contains the same address
centreSt = df[(df["ADDRESS"] == "100 Centre street")|(df["ADDRESS"] == "100 centre street") | (df["ADDRESS"] == "100 Centre Street")]

In [6]:
eightycentreSt = df[(df["ADDRESS"] == "80 Centre street")|(df["ADDRESS"] == "80 centre street")|(df["ADDRESS"] == "80 Centre Street")]

In [7]:
worth = df[(df["ADDRESS"] == "125 worth street")]

In [8]:
madison = df[(df["ADDRESS"] == "160 Madison Street")]

In [9]:
DowntownTanks = pd.concat([centreSt, eightycentreSt, worth, madison], ignore_index = True)

In [10]:
centreSt["# OF VIOLATIONS"].sum()

42

In [11]:
centreSt["# of Tanks"].sum()

34

In [12]:
eightycentreSt["# OF VIOLATIONS"].sum()

16

In [13]:
##ok I realized I have to do that for every line of data so datawrapper tooltip can show if it ever received violations between 2020 and now
df['ADDRESS_NORM'] = df['ADDRESS'].str.lower().str.strip()

In [14]:
inspection_cols = ['SEDIMENT', 'BIOLOGICAL_GROWTH', 'DEBRIS_INSECTS', 
                   'RODENT_BIRD', 'COLIFORM', 'ECOLI']

In [15]:
## create a function to summarize the violations for each address
def summarize_violations(group):
    summary = {}
    for col in inspection_cols:
        summary[f'SUM_{col}'] = 'Violation' if (group[col] == 'Violation').any() else 'Meets standards'
    return pd.Series(summary)

In [16]:
## apply it to my normalized address df
violation_summary = df.groupby('ADDRESS_NORM').apply(summarize_violations)

/var/folders/rc/n80vvwzn0cgc5d6p28yqkgtm0000gn/T/ipykernel_24027/1190434298.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  violation_summary = df.groupby('ADDRESS_NORM').apply(summarize_violations)


In [17]:
## make a df that removes duplicates
non_agg = df.drop_duplicates('ADDRESS_NORM')[['ADDRESS_NORM', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 'ZIP', '# of Tanks', 'CENSUS_TRACT', '# OF VIOLATIONS']]

In [18]:
## merge the two dfs 
summary_df_1 = violation_summary.reset_index().merge(non_agg, on='ADDRESS_NORM', how='left')

In [19]:
summary_df_1.sample(2)

,ADDRESS_NORM,SUM_SEDIMENT,SUM_BIOLOGICAL_GROWTH,SUM_DEBRIS_INSECTS,SUM_RODENT_BIRD,SUM_COLIFORM,SUM_ECOLI,ADDRESS,LATITUDE,LONGITUDE,ZIP,# of Tanks,CENSUS_TRACT,# OF VIOLATIONS
6,111 john st,Meets standards,Meets standards,Meets standards,Meets standards,Meets standards,Meets standards,111 John St,40.707671,-74.005764,10038,2,1502,1
58,60 centre street,Violation,Meets standards,Violation,Meets standards,Meets standards,Meets standards,60 centre street,40.714467,-74.002503,10007,2,29,3


In [58]:
## run the same analysis on ALL tanks to find how many tanks of the 17,000 were inspected in the last 5 years.
summary_df_1.to_csv("DIRTY_TANKS_SUMMARIZED_ALL_INSPECTIONS_WITHCOLS.csv")

In [80]:
df_ALL_1 = pd.read_excel("WATERTANKMASTER_COPY.xlsx", sheet_name = "2020-Present")

In [81]:
df_ALL_SUMMARY = df_ALL_1.drop_duplicates(subset="ADDRESS")

In [86]:
df_ALL_SUMMARY.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8559 entries, 0 to 30500
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   BOROUGH             8559 non-null   object 
 1   ADDRESS             8559 non-null   object 
 2   ZIP                 8555 non-null   float64
 3   TANK_NUM            8559 non-null   object 
 4   INSPECTION_DATE     8559 non-null   object 
 5   INTERNAL_STRUCTURE  8543 non-null   object 
 6   EXTERNAL_STRUCTURE  8557 non-null   object 
 7   SEDIMENT            8541 non-null   object 
 8   BIOLOGICAL_GROWTH   8541 non-null   object 
 9   DEBRIS_INSECTS      8541 non-null   object 
 10  RODENT_BIRD         8557 non-null   object 
 11  COLIFORM            8550 non-null   object 
 12  ECOLI               8543 non-null   object 
 13  MEET_STANDARDS      1761 non-null   object 
 14  LATITUDE            8541 non-null   float64
 15  LONGITUDE           8541 non-null   float64
 16  COMMUNITY_

In [88]:
df_ALL_SUMMARY.sample(3)

,BOROUGH,ADDRESS,ZIP,TANK_NUM,INSPECTION_DATE,INTERNAL_STRUCTURE,EXTERNAL_STRUCTURE,SEDIMENT,BIOLOGICAL_GROWTH,DEBRIS_INSECTS,RODENT_BIRD,COLIFORM,ECOLI,MEET_STANDARDS,LATITUDE,LONGITUDE,COMMUNITY_BOARD,COUNCIL_DISTRICT,CENSUS_TRACT
11285,MANHATTAN,470 WEST END AVENUE,10024.0,1,2020-07-01 00:00:00,N,N,N,N,N,N,N,N,Y,40.786770,-73.979398,7.0,6.0,171.0
7659,MANHATTAN,80 Central Park West,10023.0,1,2020-09-02 00:00:00,N,N,N,N,N,N,N,N,NaN,40.773754,-73.977767,7.0,6.0,153.0
10117,MANHATTAN,17 East 124 Street,10035.0,1,2023-06-28 00:00:00,N,N,N,N,N,N,N,N,NaN,40.805364,-73.941844,11.0,9.0,198.0


In [90]:
df_ALL_SUMMARY.to_csv("2020-present_summarizedbyaddress.csv")